In [ ]:
from src.data.fingerprints_tools.fingerprint_generator import smiles_to_fingerprint
from src.data.mgf_tools.mgf_get import *
from src.config import *
from src.utils import *

# SMILES


In [ ]:
mgf_data = r"/Users/carla/PycharmProjects/Mestrado/Transformer-Based-Models-for-Chemical-Fingerprint-Prediction/datasets/raw/cleaned_gnps_library.mgf"


In [ ]:
spectra = mgf_get_spectra(mgf_data)

In [ ]:
smiles_data = mgf_get_smiles(spectra)


In [ ]:
smiles = smiles_data['smiles']
ids = smiles_data['id']


In [ ]:
dataset = smiles_to_fingerprint(smiles_data=smiles, ids=ids, return_df=True)

In [ ]:
# smiles_dataset = 

# SPECTRA

In [ ]:
processed_spectra = mgf_deconvoluter(mgf_data=spectra, mz_vocabs=mz_vocabs, min_num_peaks=5, max_num_peaks=max_num_peaks, noise_rmv_threshold=0.01, mass_error=0.01, allowed_spectral_entropy=True, log=True)

# SPLIT

In [ ]:
from deepmol.splitters import MultiTaskStratifiedSplitter

spectrum_ids = [spec[0] for spec in processed_spectra]
filtered_smiles = smiles_data[smiles_data['spectrum id'].isin(spectrum_ids)]

In [ ]:
temp_csv = filtered_smiles[['spectrum id', 'smiles']].rename(columns={'spectrum id': 'ids'})
temp_csv.to_csv('temp_dataset.csv', index=False)

In [ ]:
from deepmol.loaders import CSVLoader
from deepmol.compound_featurization import MorganFingerprint

In [ ]:
loader = CSVLoader(dataset_path='temp_dataset.csv',
                   smiles_field='smiles', 
                   id_field='ids',
                   mode='auto')

smiles_dataset = loader.create_dataset()
smiles_dataset = MorganFingerprint().featurize(smiles_dataset)
smiles_dataset._y = smiles_dataset.X  # Fingerprints como targets

In [ ]:
train_dataset, val_dataset, test_dataset = MultiTaskStratifiedSplitter().train_valid_test_split(
    smiles_dataset, frac_train=0.8, frac_val=0.1, frac_test=0.1
)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")